In [1]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch, helpers
import requests
from IPython.display import JSON
import numpy as np
from fuzzywuzzy import fuzz
from joblib import Parallel, delayed
import pickle
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics
import math
from thefuzz import fuzz, process
import re
from IPython.display import clear_output, display
import json
from unidecode import unidecode
#from colavsim import colav_similarity, parse_doi, parse_string
import sys
from hunahpu.Similarity import parse_string, ColavSimilarity as  colav_similarity

In [ ]:
# from nltk import ngrams
# from unidecode import unidecode
# from thefuzz import fuzz, process
# from langid import classify
# from googletrans import Translator

# import re


# def __parse_string(text):
#     '''
#     This function allows to remove unneeded characters

#     Parameters
#     ----------
#     text : str
#         text to parse
#     '''
#     data = str(text).lower()
#     data = re.sub(r'<[^>]+>', '', data)
#     data = re.sub(r'[\$_\^]', '', re.sub(r'\\\w+', '', data))
#     data = unidecode(data)
#     return data

# def parse_string(text):
#     return __parse_string(text)



# def __colav_similarity(
#         paper1,
#         paper2,
#         ratio_thold=90,
#         partial_thold=95,
#         low_thold=80,
#         use_translation=False,
#         verbose=0):
#     '''
#     Utility function to avoid code duplication, public implementation below.
#     '''
#     label = False

#     title1 = paper1['title']
#     journal1 = paper1['journal']
#     year1 = paper1['year']

#     title2 = paper2['title']
#     journal2 = paper2['journal']
#     year2 = paper2['year']

#     year_check = False
#     if year1 and year2:
#         if year1 == year2:
#             year_check = True

#     journal_check = False
#     if journal1 and journal2:
#         if fuzz.partial_ratio(
#             unidecode(
#                 journal1.lower()), unidecode(
#                 journal2.lower())) > ratio_thold:
#             journal_check = True

#     length_check = False
#     if len(title1.split()) >= 1 and len(title2.split()) >= 1:
#         length_check = True

#     # Si son pocas palabras y no hay por lo menos revista o año para revisar, se descarta de uan vez
#     if length_check == False and (journal_check == False or year_check == False):
#         return label

#     if verbose == 5:
#         if journal_check:
#             print("Journals are the same")
#         if year_check:
#             print("Years are the same")

#     ratio = fuzz.ratio(title1, title2)
#     if verbose == 5:
#         print("Initial ratio: ", ratio)
#     if ratio > ratio_thold and length_check:
#         label = True
#     else:
#         title1_list = title1.split("[")
#         title2_list = title2.split("[")
#         if min([len(item) for item in title1_list]) > 10 and min([len(item) for item in title2_list]) > 10:
#             for title in title1_list:
#                 _, ratio = process.extractOne(
#                     title, title2_list, scorer=fuzz.ratio)
#                 if ratio > ratio_thold:
#                     label = True
#                     break
#             if verbose == 5:
#                 print("ratio over list: ", ratio)
#             if not label:
#                 for title in title1_list:
#                     _, ratio = process.extractOne(
#                         title, title2_list, scorer=fuzz.partial_ratio)
#                     if ratio > partial_thold:
#                         label = True
#                         break
#                     elif ratio > low_thold:
#                         if journal_check and year_check:
#                             label = True
#                             break
#                 if verbose == 5:
#                     print("partial ratio over list: ", ratio)

#     # Partial ratio section
#     if label == False:

#         ratio = fuzz.partial_ratio(title1, title2)
#         if verbose == 5:
#             print("partial ratio: ", ratio)

#         if ratio > partial_thold and length_check:
#             label = True
#         elif ratio > low_thold:
#             if journal_check and year_check:
#                 label = True

#     # Translation section
#     if label is False and use_translation:
#         lang1, _ = classify(title1)
#         lang2, _ = classify(title2)
#         if lang1 != "en" or lang2 != "en":
#             if lang1 != "en":
#                 translator = Translator()
#                 try:
#                     title1 = translator.translate(title1).text
#                 except Exception as e:
#                     if verbose == 5:
#                         print(e)
#                 if verbose == 5:
#                     print("Title 1 translated to ", title1)
#             if lang2 != "en":
#                 translator = Translator()
#                 try:
#                     title2 = translator.translate(title2).text
#                 except Exception as e:
#                     print(e)
#                 if verbose == 5:
#                     print("Title 2 translated to ", title2)

#             ratio = fuzz.ratio(title1, title2)
#             if verbose == 5:
#                 print("Ratio over translation: ", ratio)
#             if ratio > ratio_thold:
#                 label = True
#             if label is False:
#                 ratio = fuzz.partial_ratio(title1, title2)
#                 if verbose == 5:
#                     print("partial ratio over translation: ", ratio)
#                 if ratio > partial_thold:
#                     label = True
#                 elif ratio > low_thold:
#                     if journal_check and year_check:
#                         label = True

#     return label


# def colav_similarity(
#         paper1,
#         paper2,
#         ratio_thold=96,
#         partial_thold=98,
#         low_thold=76,
#         use_translation=False,
#         use_parsing=True):
#     '''
#     custom metric for similarity using multiple nested metrics from fuzzywuzzy

#     Parameters
#     ----------
#     paper1: dictionary
#         dictionary with keys title,journal and year is required
#     paper2: dictionary
#         dictionary with keys title,journal and year is required
#     ratio_thold: int
#         threshold for  ratio matric
#     partial_thold: int
#         threshold for partial ratio
#     low_thold: int
#         low threshold for ratios
#     use_translation : str
#         enable translation support
#     use_parsing: boolean
#         use parsing to remove unneeded characters
#     '''

#     if paper1['title']  is None:
#         paper1['title']  = ""
#     if paper2['title']  is None:
#         paper2['title']  = ""
#     if not isinstance(paper1['journal'], str):
#         paper1['journal'] = str(paper1['journal'])
#     if not isinstance(paper2['journal'], str):
#         paper2['journal'] = str(paper2['journal'])

#     paper1['title'] = unidecode(paper1['title'].lower())
#     paper2['title'] = unidecode(paper2['title'].lower())

#     if paper1['year']:
#         paper1['year'] = int(paper1['year'])
#     if paper2['year']:
#         paper2['year'] = int(paper2['year'])

#     label = False

#     if not use_parsing:
#         label = __colav_similarity(
#             paper1,
#             paper2,
#             ratio_thold,
#             partial_thold,
#             low_thold,
#             use_translation)
#     elif use_parsing:
#         paper1['title'] = __parse_string(paper1['title'])
#         paper2['title'] = __parse_string(paper2['title'])
#         label = __colav_similarity(
#             paper1,
#             paper2,
#             ratio_thold,
#             partial_thold,
#             low_thold,
#             use_translation)
#     return label


In [2]:
es = Elasticsearch(host="172.19.31.8",timeout=120)

In [3]:
# open a file, where you stored the pickled data
file = open('dataset_full.pkl', 'rb')
# dump information to that file
data = pickle.load(file)
# close the file
file.close()

In [4]:
len(data["p"])

101773

In [5]:
len(data["n"])

41596

In [6]:
es_index = list(es.indices.get_alias("*").keys())
print(es_index)
#es_index.remove("openalex_raw")
#es_index.remove("openalex_dfr")
es_index = "openalex_parsed"

['test_array', 'openalex_parsed', 'openalex_scratch', 'openalex_raw']


/tmp/ipykernel_984606/1562848919.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es_index = list(es.indices.get_alias("*").keys())


In [7]:
def get_oa_doi(es_reg):
    for i in es_reg["_source"]['external_ids']:
        if i['source'] == "doi":
            return i["id"]


def get_oa_doi(es_reg):
    return es_reg["_source"]['doi']


In [8]:
def search(Title, Authors:list, Source:str, Year:str, Volume:str, Issue:str, PageStart:str, PageEnd:str):
    """
    Definir el valor mínimo del _score que deseas
    Puedes ajustar este valor según tus necesidades

    """
    if not isinstance(Volume, str) or not isinstance(Volume, int):
        Volume = ""
    if not isinstance(Issue, str) or not isinstance(Issue, int):
        Issue = ""
    if not isinstance(PageStart, str) or not isinstance(PageStart, int):
        PageStart = ""
    if not isinstance(PageEnd, str) or not isinstance(PageEnd, int):
        PageEnd = ""

#    should = [      { "term": { "year": Year }},
#     should = [      { "match": { "year":  { 
#                                  "query": Year,
#                                  "boost": 16
#                               }}},
#                     { "term": { "volume": Volume }},
#                     { "term": { "issue": Issue }},
#                     { "term": { "first_page": PageStart }},
#                     { "term": { "last_page": PageEnd }}]
    
#     if isinstance(Titles, list) :
#         for title in Titles:
#             should.append(
#             { "match": { "title": {"query": title, 
#                                    "boost": 0.8,}}}
#             )
#     else:
#         print("Error, Titles should be list")
#         sys.exit(1)
    authors = []
    if isinstance(Authors, list) :
        for author in Authors:
            authors.append(
                    { "match": { "authors":  {
                                 "query": author,
                                 "operator": "AND",
                                 # "boost": 2
                              }}}            
            )
        Authors = " ".join(Authors)
    else:
        print("Error, Authors should be list")
        sys.exit(1)
#     if Source:
#         should.append({ "match": { "source":  {
#                                  "query": str(Source),
#                                  "operator": "AND",
#                                  "boost": 2
#                               }}},)
    # if not isinstance(Source, str) and not None:
    #     Source = ""
    body = {
        "query": {
            "bool": {
                "should": 
                [
                    # { "match": { "title":  {
                    #              "query": Title,
                    #              "operator": "OR"
                    #           }}},
                     #{ "match_phrase": { "title": { "query": Title, "boost": 0.8 }}},
                    { "match": { "title": { 
                                            "query": Title, 
                                            "boost": 0.8,}}},
                    #{ "match": { "authors":  Authors }},# se tienen que truncar los autores 
                    # { "match": { "authors":  {
                    #              "query": Authors,
                    #              "operator": "AND",
                    #              # "boost": 2
                    #           }}},
                    { "match": { "source":  {
                                 "query": Source,
                                 "operator": "AND",
                                 # "boost": 2
                              }}},
                    # { "match": { "year":  { # term can not be boosted, , the contribution to the score is too small, it should be boosted a lotand them it has to be match
                    #              "query": Year,
                    #              "boost": 16
                    #           }}},
                    # { "match": { "volume":  {
                    #              "query": Volume,
                    #              "boost": nterms
                    #           }}},
                    # { "match": { "issue":  {
                    #              "query": Issue,
                    #              "boost": nterms
                    #           }}},
                    # { "match": { "first_page":  {
                    #              "query": PageStart,
                    #              "boost": nterms
                    #           }}},
                    # { "match": { "last_page":  {
                    #              "query": PageEnd,
                    #              "boost": nterms
                    #           }}},
                    { "term": { "year": Year }},
                    { "term": { "volume": Volume }},
                    { "term": { "issue": Issue }},
                    { "term": { "first_page": PageStart }},
                    { "term": { "last_page": PageEnd }},
                    
                ] ,
            }
        },
        "size": 20
    }
    body["query"]["bool"]["should"].extend(authors)
    res = es.search(index=es_index, **body)
    return  res

In [ ]:
# def parse_string(string:str):
#     data = str(string).lower()
#     data = re.sub(r'<[^>]+>','',data)
#     data = unidecode(data)
#     return data    
# def parse_doi(doi):
    
#     return re.sub(r'https*\:\/\/[\w\.]+\/','',str(doi).lower())

In [9]:
%%time
#ratio_thold=90, partial_thold=90,low_thold=85) #3000 / fn = 26
#ratio_thold=95, partial_thold=90,low_thold=80 #3000 / fn = 25
#ratio_thold=90, partial_thold=90,low_thold=80 #3000 / fn = 24
#dfr fn = 47 Title, Source year 
#openalex_parsed fn =37  all fields
#openalex_parsed fn = .79  all fields all dataset
#openalex_parsed fn = 0.70 fp 6.041  all fields all dataset mode 3


def es_similarity(rec,th_high = 0, th_low = 0, mode = 0):
    res = search(parse_string(rec[0]['Title']),rec[0]['Authors'].split(",")[0:5],rec[0]['Source title'],rec[0]['Year'],
    #res = search(parse_string(rec[0]['Title'])," ".join(rec[0]['Authors'].split(",")[0:5]),rec[0]['Source title'],rec[0]['Year'],
                 rec[0]['Volume'], rec[0]['Issue'], rec[0]['Page start'], rec[0]['Page end'])
    if res["hits"]["total"]["value"] != 0:
        _score = res["hits"]["hits"][0]["_score"]
        if mode == 0: # tests mode (perfect case)
            oa_doi = get_oa_doi(res["hits"]["hits"][0]).lower()
            sc_doi = rec["DOI"].lower()
            if parse_doi(oa_doi) == parse_doi(sc_doi):
                return (True,res["hits"]["hits"][0]["_score"])
            else:
                return (False,res["hits"]["hits"][0]["_score"])           
        if mode == 1: # la salida la evalua colav similarity, solo sobre el mejor hit
            i = res["hits"]["hits"][0]
            value = colav_similarity(rec['Title']       ,  i["_source"]['title'],
                                    rec['Source title'],   i["_source"]["source"],
                                    rec['Year']        ,   i["_source"]['year'],
                                    ratio_thold=95, partial_thold=95,low_thold=82)
            return (value,i["_score"])
#         if mode == 2: # después de los cortes la salida la evalua colav similarity
#             if _score >= th_high:
#                 return (True,_score)
#             elif _score <= th_low:
#                 return (False,_score)                
#             else:
#                 for i in res["hits"]["hits"]:
#                     paper1={}
#                     paper1["title"] = parse_string(rec[0]['Title'])
#                     paper1["journal"] = str(rec[0]['Source title'])
#                     paper1["year"] = rec[0]['Year']
                    
#                     paper2 = {}
#                     paper1["title"] = parse_string(i["_source"]['title'])
#                     paper1["journal"] = str(i["_source"]["source"])
#                     paper1["year"] = i["_source"]['year']
                    
#                     value = colav_similarity(paper1, paper2   ,
#                                             ratio_thold=99, partial_thold=98,low_thold=82)
#                     if value:
#                         return (True, i["_score"])
#                 return (False, i["_score"])

        if mode == 3: # la salida la evalua colav similarity sobre los 20 primeros hits
            for i in res["hits"]["hits"]:
                paper1={}
                paper1["title"] = parse_string(rec[0]['Title'])
                paper1["journal"] = str(rec[0]['Source title'])
                paper1["year"] = rec[0]['Year']

                paper2 = {}
                paper2["title"] = parse_string(i["_source"]['title'])
                paper2["journal"] = str(i["_source"]["source"])
                paper2["year"] = i["_source"]['year']                
                value = colav_similarity(paper1, paper2,
                # value = colav_similarity(rec[0]['Title']       ,  i["_source"]['title'],
                #         rec[0]['Source title'],   i["_source"]["source"],
                #         rec[0]['Year']        ,   i["_source"]['year'],
                       ratio_thold=90, partial_thold=92,low_thold=81)
#                       ratio_thold=99, partial_thold=98,low_thold=82)
                if value:
                    return (True, i["_score"])
                
            return (False,_score)
        
        if mode == 5:
            _score = res["hits"]["hits"][0]["_score"]
            return (rec[1],_score)
        if mode == 6:
            _score = res["hits"]["hits"][0]["_score"]
            if _score >= th_high:
                return (True,_score)
            else:
                return (False,_score)
#             iss1 = rec['Issue']
#             iss2 = i["_source"]["issue"]
#             if iss1 and iss2:
#                 if iss1 != iss2:
#                     return (False, i["_score"])
    
            # v1 = rec['Volume']
            # v2 = i["_source"]["volume"]
            # if v1 and v2:
            #     if v1 != v2:
            #         return (False, i["_score"])
            
            return (True, i["_score"])
        print("ERROR: method no valid, select 0,1,2 or 4")
        sys.exit(1)
    else:
        return (False,0)
scopus = []
np.random.shuffle(data["p"])
np.random.shuffle(data["n"])
for i in data["p"]:#[0:10001]:
    scopus.append((i,True))
for i in data["n"]:#[0:10000]:
    scopus.append((i,False))
np.random.shuffle(scopus) #shuffle is inplace
#th_high=50  0.57 99,98,82
th_high=500
th_low =0
mode = 3
results = Parallel(n_jobs=72,backend='threading',verbose=2)(delayed(es_similarity)(rec,th_high, th_low, mode) for rec in scopus)

[Parallel(n_jobs=72)]: Using backend ThreadingBackend with 72 concurrent workers.
[Parallel(n_jobs=72)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=72)]: Done 221 tasks      | elapsed:    1.0s
[Parallel(n_jobs=72)]: Done 504 tasks      | elapsed:    2.2s
[Parallel(n_jobs=72)]: Done 869 tasks      | elapsed:    3.8s
[Parallel(n_jobs=72)]: Done 1314 tasks      | elapsed:    5.7s
[Parallel(n_jobs=72)]: Done 1841 tasks      | elapsed:    7.9s
[Parallel(n_jobs=72)]: Done 2448 tasks      | elapsed:   10.5s
[Parallel(n_jobs=72)]: Done 3137 tasks      | elapsed:   13.2s
[Parallel(n_jobs=72)]: Done 3906 tasks      | elapsed:   16.3s
[Parallel(n_jobs=72)]: Done 4757 tasks      | elapsed:   19.8s
[Parallel(n_jobs=72)]: Done 5688 tasks      | elapsed:   23.6s
[Parallel(n_jobs=72)]: Done 6701 tasks      | elapsed:   28.2s
[Parallel(n_jobs=72)]: Done 7794 tasks      | elapsed:   33.0s
[Parallel(n_jobs=72)]: Done 8969 tasks      | elapsed:   37.9s
[Parallel(n_jobs=72)]: Done 10224 tasks 

RequestError: RequestError(400, 'x_content_parse_exception', "[1:404] Non-standard token 'NaN': enable `JsonReadFeature.ALLOW_NON_NUMERIC_NUMBERS` to allow\n at [Source: (org.elasticsearch.common.io.stream.ByteBufferStreamInput); line: 1, column: 404]")

In [10]:
#>tp, tn, fp, fn
actual = []
predicted = []
for i in range(len(results)):
    if results[i] is not None:
        actual.append(scopus[i][1])
        predicted.append(results[i][0])


NameError: name 'results' is not defined

In [ ]:
confusion_matrix = metrics.confusion_matrix(actual, predicted)
confusion_matrix = np.flip(confusion_matrix)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [True, False])

cm_display.plot()
plt.show()

In [ ]:
fn = confusion_matrix[0][1]*100/(confusion_matrix[0][0]+confusion_matrix[0][1])
fn

In [ ]:
fp = confusion_matrix[1][0]*100/(confusion_matrix[1][1]+confusion_matrix[1][0])
fp

In [ ]:
acc = metrics.accuracy_score(actual,predicted)
pre = metrics.precision_score(actual,predicted)
rec = metrics.recall_score(actual,predicted)
f1  = metrics.f1_score(actual,predicted)

print(f"Accuracy  = {acc:.4f}")
print(f"Precision = {pre:.4f}")
print(f"Recall    = {rec:.4f}")
print(f"F1        = {f1:.4f}")


In [ ]:
score_p = []
score_n = []
for i in results:
    if i[0]:
        score_p.append(i[1])
    else:
        score_n.append(i[1])  

In [ ]:
np.mean(score_p)

In [ ]:
len(score_p)

In [ ]:
len(score_n)

In [ ]:
f_score_p = []
f_score_n = []

for i in score_p:
    if i<1000:
        f_score_p.append(i)

for i in score_n:
    if i<1000:
        f_score_n.append(i)

In [ ]:
f_score_p = np.array(f_score_p)
f_score_n = np.array(f_score_n)


In [ ]:
f_score_p.max()


In [ ]:
# Obtiene la información del histograma
valores_p, bordes_p = np.histogram(f_score_p, bins=100, density=True)
centros_p = 0.5 * (bordes_p[:-1] + bordes_p[1:])

# Encuentra el valor máximo en el eje Y
valor_maximo_y_p = max(valores_p)

# Encuentra el valor en el eje X para el cual Y es máximo
valor_maximo_x_p = centros_p[np.argmax(valores_p)]
valor_maximo_x_p

In [ ]:
# Obtiene la información del histograma
valores_n, bordes_n = np.histogram(f_score_n, bins=100, density=True)
centros_n = 0.5 * (bordes_n[:-1] + bordes_n[1:])

# Encuentra el valor máximo en el eje Y
valor_maximo_y_n = max(valores_n)

# Encuentra el valor en el eje X para el cual Y es máximo
valor_maximo_x_n = centros_p[np.argmax(valores_n)]
valor_maximo_x_n

In [ ]:
n_bins = 100


plt.figure(figsize=(8,6))
plt.hist(f_score_p, bins=100, alpha=0.5, label="data1", density=False)
plt.hist(f_score_n, bins=100, alpha=0.5, label="data1", density=False)
# Dibuja una línea vertical en el punto máximo
plt.axvline(x=valor_maximo_x_p, color='r', linestyle='--', label=f'Máximo en X: {valor_maximo_x_p:.2f}')
plt.axvline(x=valor_maximo_x_n, color='r', linestyle='--', label=f'Máximo en X: {valor_maximo_x_n:.2f}')

True

In [ ]:
scopus[0]

In [ ]:
rec = scopus[0:2]
title = [parse_string(rec[0][0]['Title']), parse_string(rec[1][0]['Title'])]
res = search(title,", ".join(rec[0][0]['Authors'].split(",")[0:5]),rec[0][0]['Source title'],rec[0][0]['Year'],
                 rec[0][0]['Volume'], rec[0][0]['Issue'], rec[0][0]['Page start'], rec[0][0]['Page end'])

In [ ]:
res